# Motivation : How to find the best system prompt for a given task.,lets use role play to find the best system prompt.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")



In [2]:
start_prompt="""You are a routing assistant, route the user to the best options among HR,Finance,IT,Sales,Marketing,Engineering.Only return the department name.(nothing else)"""

In [12]:
start_prompt="""You are a routing assistant, route the user to the best options among HR,Finance,IT,Sales,Marketing,Engineering.Only return the department name.(nothing else)"""

class RolePlay:
    def __init__(self,llm_actor,llm_prompt_director,llm_checker,start_prompt):
        self.llm_actor = llm_actor
        self.llm_prompt_director = llm_prompt_director
        self.llm_checker = llm_checker
        self.start_prompt = start_prompt
        self.test_data = [
            ("I need help with my salary slip", "Finance"),
            ("My laptop is not working", "IT"), 
            ("Looking to join the company", "HR"),
            ("Need to discuss the new product features", "Engineering"),
            ("Want to discuss the marketing campaign", "Marketing"),
            ("Need help with customer queries", "Sales"),
            ("Having issues with my benefits", "HR"),
            ("Need to discuss revenue targets", "Sales"),
            ("Server is down", "IT"),
            ("Need budget approval", "Finance")
        ]

    def play(self, system_prompt, user_prompt):
        import time
        best_prompt = system_prompt
        best_correct_count = 0
        best_results = []
        
        print("Starting role play iterations...")
        
        for attempt in range(10):  # Limit to 10 tries
            print(f"\nAttempt {attempt + 1}:")
            correct_count = 0
            results = []
            
            # Test current prompt on all test cases
            print("\nTesting current prompt on test cases:")
            for test_input, expected in self.test_data:
                print(f"\nTest input: {test_input}")
                messages = [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": test_input}
                ]
                response = self.llm_actor.invoke(messages)
                time.sleep(30)  # 2 second delay after API call
                print(f"Response: {response.content}")
                print(f"Expected: {expected}")
                
                # Clean response - remove whitespace and convert to uppercase for comparison
                cleaned_response = response.content.strip().upper()
                cleaned_expected = expected.strip().upper()
                is_correct = cleaned_response == cleaned_expected
                results.append((test_input, response, expected, str(is_correct)))
                print(f"Correct: {is_correct}")
                
                if is_correct:
                    correct_count += 1

            print(f"\nCorrect answers this attempt: {correct_count}/{len(self.test_data)}")

            # Update best results if current attempt is better
            if correct_count > best_correct_count:
                print("\nNew best score achieved!")
                best_correct_count = correct_count
                best_prompt = system_prompt
                best_results = results

            # If all correct, we're done
            if correct_count == len(self.test_data):
                print("\nPerfect score achieved! Ending iterations.")
                break

            # Get improved prompt for next iteration
            print("\nGetting improved prompt...")
            messages = [
                {"role": "user", "content": f"""Current prompt: {system_prompt}
                Test results: {results}
                Please improve the prompt to better handle the failed cases.
                Return only the improved prompt."""}
            ]
            response = self.llm_prompt_director.invoke(messages)
            time.sleep(30)  # 2 second delay after API call
            system_prompt = response.content
            print(f"New prompt: {system_prompt}")

        print("\nFinal Results:")
        print(f"Best prompt: {best_prompt}")
        print(f"Best score: {best_correct_count}/{len(self.test_data)}")
        print("\nDetailed results:")
        for result in best_results:
            print(f"Input: {result[0]}")
            print(f"Response: {result[1].content}")
            print(f"Expected: {result[2]}")
            print(f"Correct: {result[3]}\n")

        return best_prompt, best_results, best_correct_count


In [13]:
role_play = RolePlay(llm,llm,llm,start_prompt)
role_play.play(start_prompt,start_prompt)


Starting role play iterations...

Attempt 1:

Testing current prompt on test cases:

Test input: I need help with my salary slip
Response: Finance
Expected: Finance
Correct: True

Test input: My laptop is not working
Response: IT
Expected: IT
Correct: True

Test input: Looking to join the company
Response: Please provide more information about your skills and interests.
Expected: HR
Correct: False

Test input: Need to discuss the new product features
Response: Marketing
Expected: Engineering
Correct: False

Test input: Want to discuss the marketing campaign
Response: Marketing
Expected: Marketing
Correct: True

Test input: Need help with customer queries
Response: Sales
Expected: Sales
Correct: True

Test input: Having issues with my benefits
Response: HR
Expected: HR
Correct: True

Test input: Need to discuss revenue targets
Response: Sales
Expected: Sales
Correct: True

Test input: Server is down
Response: IT
Expected: IT
Correct: True

Test input: Need budget approval
Response: Fina

('You are a routing assistant. Route the user to the best option among HR, Finance, IT, Sales, Marketing, and Engineering. Prioritize accuracy; if uncertain, respond with "Unclear" instead of a department.  Consider the context of the request, and the potential involvement of multiple departments. If multiple departments might be involved, carefully analyze the request to determine the most likely *primary* department.  Prioritize requests that are specific and actionable over those that are vague or broad.  Consider the following departmental responsibilities:\n\n* **HR:** Hiring, benefits, employee relations, payroll (excluding specific salary inquiries).\n* **Finance:** Budgeting, financial reporting, accounts payable/receivable, expense reports, salary slip issues.\n* **IT:** Hardware/software issues, network problems, server issues.\n* **Sales:** Customer queries, revenue targets, sales reports.\n* **Marketing:** Marketing campaigns, brand management, market research.\n* **Enginee